<a href="https://colab.research.google.com/github/Fishing-oboro/NLP-learn/blob/main/index/Doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doc2vec (Paragraph Vector)
---
カウントベースの手法によって、文章ベクトルの獲得が可能である。しかし、その手法は単語の語順や意味を表現することができないという欠点を持つ。Doc2vecでは、その欠点を克服するためにword2vecの手法を文章ベクトルの獲得に取り入れている。  
これには、PV-DBOWとPV-DMという二つの手法があり、word2vecで用いられる2つの手法に類似している。
## Doc2vecの特徴
---
1. 可変長の文章から固定長の文章ベクトルを得ることができる。
2. word2vecと同様に単語を予測する過程でベクトルを得る。
3. word2vecとほぼ同一のニューラルネットワークを用いて学習する。



## PV-DM(分散記憶モデル)
---
この手法は、**CBOWと同様に周辺単語(コンテキスト)から中心単語(ターゲット)**を推測することを目的としたニューラルネットワークを活用する。入力層を**周辺単語ベクトル＋文章ベクトル**とすることで文章全体を加味したベクトルを得ることができる。

#### NNの構造
---
![](https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/image/PV%E3%83%BCDM.jpg)
1. 入力層:コンテキストに重みWin、文章IDに重みDinを使って低次元のベクトルに変換
2. 中間層:複数のベクトルを結合(和、平均など)
3. 出力層:重みを使って元の次元のベクトルに変換し、softMax関数で確率化。
4. 誤差逆伝搬:正解ラベルと確率の誤差が小さくなるように重みDinを更新。

#### PV-DMの特徴
---
周辺単語をベクトル化する際にWinを必要とするため、あらかじめ文章中の全単語の単語ベクトルを手に入れておく必要がある。また、文章ベクトルはDinから得ることができ、word2vecにおけるWinと同様のものである。

## PV-DBOW(分散BOWモデル)
--- 

この手法は、**文書IDから文中の単語(ターゲット)**を推測することを目的としたニューラルネットワークを活用する。入力層が文章IDのみであるため、**単語ベクトルを必要としない**。  
しかし、語順を無視して文中の単語を予測するため、文脈を加味しないベクトルになる。
#### NNの構造 
---

![](https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/image/PV-DBOW.jpg)

1. 入力層:文章IDに重みDinを使って低次元のベクトルに変換
2. 中間層:ベクトルを分割
3. 出力層:重みを使って元の次元のベクトルに変換し、softMax関数で確率化。
4. 誤差逆伝搬:正解ラベルと確率の誤差が小さくなるように重みDinを更新。


## 実装例
---


In [1]:
# mecabに必要なライブラリ取得
!apt-get install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3
!ln -s /etc/mecabrc /usr/local/etc/mecabrc # シンボリックリンクの作成

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
0 upgraded, 8 newly installed, 0 to remove and 13 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 277 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd6

In [2]:
# テキストの取得
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/documents.txt
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/titles.txt

--2021-02-26 12:48:47--  https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/documents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34404 (34K) [text/plain]
Saving to: ‘documents.txt’

documents.txt       100%[===================>]  33.60K  --.-KB/s    in 0.002s  

2021-02-26 12:48:48 (15.5 MB/s) - ‘documents.txt’ saved [34404/34404]

--2021-02-26 12:48:48--  https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/titles.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 605 [t

In [12]:
import numpy as np
import MeCab
import sys
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from collections import OrderedDict

# input_text = open('./titles.txt', 'r').read()
input_text = open('./documents.txt', 'r').read()
documents = input_text.split("|")

def words(text):
    """
        文章から単語を抽出
    """
    out_words = []
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)

    while node:
        word_type = node.feature.split(",")[0]
        if word_type in ["名詞"]:
            out_words.append(node.surface)
        node = node.next
    return out_words



# 学習データとなる各文書
training_docs = []
for i, document in enumerate(documents):
    training_docs.append(TaggedDocument(words=words(document), tags=['doc' + str(i + 1)]))

# min_count=1:最低1回出現した単語を学習に使用
# dm=0: 学習モデル=DBOW
model = Doc2Vec(documents=training_docs)

tags = OrderedDict() #辞書の繰り返し時による順番を保つ
tag_list = (('doc1', "記事A"), ('doc2', "記事B"), ('doc3', "記事C"), ('doc4', "記事D"), ('doc5', "記事E"), ('doc6', "記事F"))
dic = OrderedDict(tag_list)
tags.update(dic)

for k, v in tags.items():
    print("[" + v + "]")
    for items in model.docvecs.most_similar(k):
        print("\t" + tags[items[0]] + " : "+ str(items[1]))

[記事A]
	記事D : 0.8647538423538208
	記事B : 0.8570315837860107
	記事E : 0.8542587757110596
	記事F : 0.8050104975700378
	記事C : 0.3215709328651428
[記事B]
	記事D : 0.9421840906143188
	記事E : 0.9203145503997803
	記事F : 0.886698842048645
	記事A : 0.8570316433906555
	記事C : 0.3683288097381592
[記事C]
	記事B : 0.3683288097381592
	記事A : 0.3215709626674652
	記事E : 0.30772584676742554
	記事D : 0.29134976863861084
	記事F : 0.2839837074279785
[記事D]
	記事B : 0.9421840906143188
	記事E : 0.9079012870788574
	記事A : 0.8647538423538208
	記事F : 0.8631159067153931
	記事C : 0.29134976863861084
[記事E]
	記事B : 0.9203144311904907
	記事D : 0.9079012870788574
	記事F : 0.8628782629966736
	記事A : 0.8542588353157043
	記事C : 0.30772584676742554
[記事F]
	記事B : 0.886698842048645
	記事D : 0.8631159663200378
	記事E : 0.8628782629966736
	記事A : 0.8050106167793274
	記事C : 0.2839837074279785
